# 0. Import libs, test GPU

In [1]:
!pip install transformers
!pip install wget

     |████████████████████████████████| 1.4MB 8.6MB/s 
     |████████████████████████████████| 2.9MB 26.2MB/s 
     |████████████████████████████████| 890kB 30.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=ae4804f733de1d3c01e3c5a8acf3b64872544b087332ea9cf08ff1f16d3c6b4d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9681 sha256=5c28389f0b9da08a006728c3483f0319685ef1db832765d8c77c6bc74fdf99c0
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [2]:
import torch
import wget
import os
import pandas as pd
from transformers import BertTokenizer, AutoTokenizer, AutoModel
from transformers import BertForSequenceClassification, AdamW, BertConfig 
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import get_linear_schedule_with_warmup
import numpy as np
import time
import datetime
import random
import plotly.express as px
import pandas as pd
from sklearn.metrics import matthews_corrcoef, precision_score, recall_score

In [3]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


# 1. Data load

In [4]:
# print('Downloading dataset')
# # The URL for the dataset zip file.
# url = 'https://nyu-mll.github.io/CoLA/cola_public_1.1.zip'
# # Download the file (if we haven't already)
# if not os.path.exists('./cola_public_1.1.zip'):
#     wget.download(url, './cola_public_1.1.zip')
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
# Unzip the dataset (if we haven't already)
# if not os.path.exists('./cola_public/'):
#     !unzip cola_public_1.1.zip

In [5]:
# Load the dataset into a pandas dataframe.
# df = pd.read_csv("./cola_public/raw/in_domain_train.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])
df = pd.read_csv('gdrive/MyDrive/Биоинформатика/ИБ/Проект/Data/3_TRAIN_articles.csv').fillna('')
df['sentence'] = df.title + '. ' + df.abstract
# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))
# Display 10 random rows from the data.
df.sample(10)

Number of training sentences: 31,613



,pubmed_id,title,abstract,label,sentence
26300,23815441,Natural ventilation reduces high TB transmissi...,BACKGROUND: Transmission of drug susceptible a...,0,Natural ventilation reduces high TB transmissi...
24100,17583076,Multidisciplinarism in psychiatry,,0,Multidisciplinarism in psychiatry.
25362,16370954,Allergic reactions to beta-lactams,Allergy to beta-lactam antibiotics is the most...,0,Allergic reactions to beta-lactams. Allergy to...
24382,29878318,Transforming growth factor-beta signal regulat...,"During gastrulation, one of the most important...",0,Transforming growth factor-beta signal regulat...
21139,21890677,Identification and characterization of Sclerot...,Numerous studies have shown both the detriment...,0,Identification and characterization of Sclerot...
28053,6627347,Reparative growth in the rat thyroid,This study was designed to investigate the mit...,0,Reparative growth in the rat thyroid. This stu...
21751,32021497,The Diagnostic Value of Neuron-Specific Enolas...,Background: The present study was conducted to...,0,The Diagnostic Value of Neuron-Specific Enolas...
24423,115274,Hemorrhage with pelvic fractures: efficacy of ...,To determine the effectiveness of angiography ...,0,Hemorrhage with pelvic fractures: efficacy of ...
826,32433614,Structural basis for catalysis and substrate s...,As members of the membrane-bound O-acyltransfe...,1,Structural basis for catalysis and substrate s...
17992,27723895,Effects of Long-term Serial Passaging on the C...,Purpose: Development of liver metastasis remai...,0,Effects of Long-term Serial Passaging on the C...


In [6]:
# Get the lists of sentences and their labels.
sentences = df.sentence.values
labels = df.label.values

# 2. Data Tokenization & Input Formatting

In [7]:
# Load the RoBERTa tokenizer.
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenizer = AutoTokenizer.from_pretrained('allenai/biomed_roberta_base', do_lower_case=True)

In [8]:
input_ids = []
for sent in sentences:
    encoded_sent = tokenizer.encode(
                        sent,                      
                        add_special_tokens = True,
                   )
    
    input_ids.append(encoded_sent)

print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:  Human alpha 2-macroglobulin gene is located on chromosome 12. A cDNA clone encoding amino acids 809-1451 of the protease inhibitor alpha 2-macroglobulin has been isolated from an adult human liver cDNA library. This cDNA was used to examine DNA samples prepared from a panel of human-mouse somatic cell hybrids with different numbers and combinations of human chromosomes for the presence of the human alpha 2-macroglobulin gene. The cosegregation of this gene and chromosome 12 in the cell hybrid panel indicated that the alpha 2-macroglobulin structural gene (designated A2M) is on human chromosome 12.
Token IDs: [0, 33837, 32756, 132, 12, 15439, 10191, 462, 2413, 27377, 10596, 16, 2034, 15, 44198, 316, 4, 83, 740, 39287, 39511, 45278, 37788, 27976, 290, 3546, 12, 1570, 4708, 9, 5, 27067, 3175, 34496, 32756, 132, 12, 15439, 10191, 462, 2413, 27377, 34, 57, 8067, 31, 41, 4194, 1050, 12581, 740, 39287, 5560, 4, 152, 740, 39287, 21, 341, 7, 10154, 5708, 7931, 2460, 31, 10, 2798, 9, 

In [10]:
print('Max sentence length: ', max([len(sen) for sen in input_ids]))
print('Mean sentence length: ', np.mean([len(sen) for sen in input_ids]))
print('Median sentence length: ', np.median([len(sen) for sen in input_ids]))

Max sentence length:  1552
Mean sentence length:  277.4403568152342
Median sentence length:  281.0


In [11]:
# Set the maximum sequence length.
MAX_LEN = 300
print('Padding/truncating all sentences to {} values...'.format(MAX_LEN))
print('Padding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

# input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
#                           value=0, truncating="post", padding="post")
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post", value=tokenizer.pad_token_id)

for i in range(len(input_ids)):
    input_ids[i][1:][input_ids[i][1:] == 0] = tokenizer.pad_token_id

Padding/truncating all sentences to 300 values...
Padding token: "<pad>", ID: 1


In [12]:
input_ids[0]

array([    0, 33837, 32756,   132,    12, 15439, 10191,   462,  2413,
       27377, 10596,    16,  2034,    15, 44198,   316,     4,    83,
         740, 39287, 39511, 45278, 37788, 27976,   290,  3546,    12,
        1570,  4708,     9,     5, 27067,  3175, 34496, 32756,   132,
          12, 15439, 10191,   462,  2413, 27377,    34,    57,  8067,
          31,    41,  4194,  1050, 12581,   740, 39287,  5560,     4,
         152,   740, 39287,    21,   341,     7, 10154,  5708,  7931,
        2460,    31,    10,  2798,     9,  1050,    12, 45052, 16487,
        5183,  3551, 34297,    19,   430,  1530,     8, 21092,     9,
        1050, 44253,    13,     5,  2621,     9,     5,  1050, 32756,
         132,    12, 15439, 10191,   462,  2413, 27377, 10596,     4,
          20,   740,  3876, 46607,     9,    42, 10596,     8, 44198,
         316,    11,     5,  3551,  9284,  2798,  4658,    14,     5,
       32756,   132,    12, 15439, 10191,   462,  2413, 27377,  9825,
       10596,    36,

In [13]:
# Create attention masks
attention_masks = []

for sent in input_ids:
    
    # Create the attention mask.
    att_mask = [int(token_id != tokenizer.pad_token_id) for token_id in sent]
    
    # Store the attention mask for this sentence.
    attention_masks.append(att_mask)

In [14]:
# Train/Test Split
# Use 90% for training and 10% for validation.
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=42, test_size=0.1)

# Do the same for the masks.
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels, random_state=42, test_size=0.1)

In [15]:
# Convert all inputs and labels into torch tensors 
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [16]:
# Create DataLoaders
batch_size = 16
# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
# Create the DataLoader for our validation set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

# 3. Train

In [ ]:
# Load the pretrained RoBERTa model with a single linear classification layer on top. 

# model = BertForSequenceClassification.from_pretrained(
#     "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
#     num_labels = 2, # The number of output labels--2 for binary classification.
#                     # You can increase this for multi-class tasks.   
#     output_attentions = False, # Whether the model returns attentions weights.
#     output_hidden_states = False, # Whether the model returns all hidden-states.
# )

model = BertForSequenceClassification.from_pretrained(
    'allenai/biomed_roberta_base',
    num_labels = 2, 
    output_attentions = False,
    output_hidden_states = False,
)

model.cuda()

In [18]:
# Get all of the model's parameters

params = list(model.named_parameters())
print('The RoBERTa model has {:} different named parameters.\n'.format(len(params)))
print('==== Embedding Layer ====\n')
for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== First Transformer ====\n')
for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== Output Layer ====\n')
for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The RoBERTa model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (50265, 768)
bert.embeddings.position_embeddings.weight                (514, 768)
bert.embeddings.token_type_embeddings.weight                (1, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias            

In [19]:
# Optimiser
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8
                )

# Number of training epochs (authors recommend between 2 and 4)
epochs = 20
# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs
# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [20]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [21]:
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
# Store the average loss after each epoch so we can plot them.
loss_values = []
# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    # Measure how long the training epoch takes.
    t0 = time.time()
    # Reset the total loss for this epoch.
    total_loss = 0
    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()
    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):
        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        
        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]
        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()
        # Update the learning rate.
        scheduler.step()
    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.
    print("")
    print("Running Validation...")
    t0 = time.time()
    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()
    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        
            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy
        # Track the number of batches
        nb_eval_steps += 1
    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
print("")
print("Training complete!")


======== Epoch 1 / 20 ========
Training...
  Batch    40  of  1,779.    Elapsed: 0:00:40.
  Batch    80  of  1,779.    Elapsed: 0:01:18.
  Batch   120  of  1,779.    Elapsed: 0:01:57.
  Batch   160  of  1,779.    Elapsed: 0:02:36.
  Batch   200  of  1,779.    Elapsed: 0:03:15.
  Batch   240  of  1,779.    Elapsed: 0:03:54.
  Batch   280  of  1,779.    Elapsed: 0:04:33.
  Batch   320  of  1,779.    Elapsed: 0:05:11.
  Batch   360  of  1,779.    Elapsed: 0:05:50.
  Batch   400  of  1,779.    Elapsed: 0:06:29.
  Batch   440  of  1,779.    Elapsed: 0:07:08.
  Batch   480  of  1,779.    Elapsed: 0:07:47.
  Batch   520  of  1,779.    Elapsed: 0:08:26.
  Batch   560  of  1,779.    Elapsed: 0:09:04.
  Batch   600  of  1,779.    Elapsed: 0:09:43.
  Batch   640  of  1,779.    Elapsed: 0:10:22.
  Batch   680  of  1,779.    Elapsed: 0:11:01.
  Batch   720  of  1,779.    Elapsed: 0:11:39.
  Batch   760  of  1,779.    Elapsed: 0:12:18.
  Batch   800  of  1,779.    Elapsed: 0:12:57.
  Batch   840  o

In [24]:
import pickle as pk

model_output_path = os.path.join('gdrive/MyDrive/Биоинформатика/ИБ/Проект/', 'model.pkl')
torch.save(model.state_dict(), model_output_path, pickle_protocol=pk.HIGHEST_PROTOCOL)

model_output_path_tar = os.path.join('gdrive/MyDrive/Биоинформатика/ИБ/Проект/', 'model.tar')
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss_values': loss_values,
            }, model_output_path_tar, pickle_protocol=pk.HIGHEST_PROTOCOL)

model_output_path_all_model = os.path.join('gdrive/MyDrive/Биоинформатика/ИБ/Проект/', 'model.pth')
torch.save(model, model_output_path_all_model)

In [25]:
f = pd.DataFrame(loss_values)
f.columns=['Loss']
fig = px.line(f, x=f.index, y=f.Loss)
fig.update_layout(title='Training loss of the Model',
                   xaxis_title='Epoch',
                   yaxis_title='Loss')
fig.show()

# Test

In [38]:
# Load the dataset into a pandas dataframe.
df = pd.read_csv("gdrive/MyDrive/Биоинформатика/ИБ/Проект/Data/3_TEST_articles.csv").fillna('')
df['sentence'] = df.title + '. ' + df.abstract
# Report the number of sentences.
print('Number of test sentences: {:,}\n'.format(df.shape[0]))
# Create sentence and label lists
sentences = df.sentence.values
labels = df.label.values
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
# For every sentence...
for sent in sentences:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                   )
    
    input_ids.append(encoded_sent)
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, 
                          dtype="long", truncating="post", padding="post", value=tokenizer.pad_token_id)
for i in range(len(input_ids)):
    input_ids[i][1:][input_ids[i][1:] == 0] = tokenizer.pad_token_id

# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i != tokenizer.pad_token_id) for i in seq]
  attention_masks.append(seq_mask) 
# Convert to tensors.
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)
# Set the batch size.  
batch_size = 32  
# Create the DataLoader.
prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

Number of test sentences: 2,000



In [39]:
# Prediction on test set
print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))
# Put model in evaluation mode
model.eval()
# Tracking variables 
predictions , true_labels = [], []
# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)
  logits = outputs[0]
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)
print('DONE.')

Predicting labels for 2,000 test sentences...
DONE.


In [40]:
print('Positive samples: %d of %d (%.2f%%)' % (df.label.sum(), len(df.label), (df.label.sum() / len(df.label) * 100.0)))

Positive samples: 826 of 2000 (41.30%)


In [41]:
matthews_set = []
# Evaluate each test batch using Matthew's correlation coefficient
print('Calculating Matthews Corr. Coef. for each batch...')
# For each input batch...
for i in range(len(true_labels)):
  
  # The predictions for this batch are a 2-column ndarray (one column for "0" 
  # and one column for "1"). Pick the label with the highest value and turn this
  # in to a list of 0s and 1s.
  pred_labels_i = np.argmax(predictions[i], axis=1).flatten()
  
  # Calculate and store the coef for this batch.  
  matthews = matthews_corrcoef(true_labels[i], pred_labels_i)                
  matthews_set.append(matthews)

Calculating Matthews Corr. Coef. for each batch...


In [42]:
# Combine the predictions for each batch into a single list of 0s and 1s.
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
# Combine the correct labels for each batch into a single list.
flat_true_labels = [item for sublist in true_labels for item in sublist]
# Calculate the MCC
mcc = matthews_corrcoef(flat_true_labels, flat_predictions)
precision = precision_score(flat_true_labels, flat_predictions)
recall = recall_score(flat_true_labels, flat_predictions)
print('MCC: %.3f' % mcc)
print('Precision: %.3f' % precision)
print('Recall: %.3f' % recall)

MCC: 0.859
Precision: 0.902
Recall: 0.935
